<a href="https://colab.research.google.com/github/fabirian/Trabajo-Grado/blob/main/Model_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

CARGA E IMPORTACION DE LAS LIBRERIAS NECESARIAS

In [ ]:

import json
import os
import pickle
import random
import time
from datetime import datetime

import cv2
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
from fpdf import FPDF
from gensim.models import Word2Vec
from keras import Model
from keras.layers import (Concatenate, Conv2D, Dense, Dropout, Embedding, Input, LSTM)
import nltk
from nltk.translate.bleu_score import SmoothingFunction, sentence_bleu
from PIL import Image
from rouge import Rouge
from sklearn.model_selection import train_test_split
from keras.applications import DenseNet121, InceptionV3
from keras.preprocessing import image
from keras.applications.inception_v3 import preprocess_input
from keras.preprocessing.image import img_to_array, load_img
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from tqdm import tqdm
from keras.initializers import GlorotUniform, HeNormal
from keras.layers import BatchNormalization
from keras import regularizers
from keras.layers import GRU
from pathlib import Path
from tensorflow.keras.layers import LayerNormalization
from tensorflow.keras.layers import Reshape
from tensorflow.keras.layers import MultiHeadAttention
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

PROCESAMIENTO DE IMAGENES

CARGA DEL MODELO USADO PARA EXTRACCION DE CARACTERISTICAS

In [ ]:
# Usar el modelo preentrenado InceptionV3

model = InceptionV3(include_top=False, weights='imagenet')

# Definiendo la capa de entrada y la capa de salida
input_layer = model.input
output_layer = model.layers[-2].output

# Creando el modelo
image_model = tf.keras.Model(input_layer, output_layer)

87910968/87910968 [==============================] - 0s 0us/step


FUNCION DE PROCESAMIENTO DE IMAGENES

In [ ]:
def preprocess(image_path, path, target_size=(350, 350)):
    '''
    Entrada -- image_path: ruta de la imagen, path: directorio base de la imagen
    Salida -- tensor de imagen int32
    '''

    # Unir el directorio base con el nombre de la imagen
    full_image_path = os.path.join(path, image_path)

    img = image.load_img(full_image_path, target_size=target_size)

    # Convertir la imagen a un arreglo y preprocesar.
    img_array = image.img_to_array(img)
    img_array = preprocess_input(img_array)  # Utiliza la función de preprocesamiento de InceptionV3.

    # Añadir una dimensión para formar un lote de tamaño 1.
    img_batch = np.expand_dims(img_array, axis=0)

    # Cargar el modelo y predecir.
    features = model.predict(img_batch)

    features = tf.cast(features, tf.float32)

    return features

In [3]:
# create the tensor using train and validation data and store it in list

path = '/home/gico/Neumonia/Img_Completo/'

# Cargos los distintos tipos de dataset

# Data Normal
#train = pd.read_csv('/home/gico/Neumonia/Archivos_csv/DatosModelo/DatosConLimpieza/train_data_normalizada_sindup.csv')
#validation = pd.read_csv('/home/gico/Neumonia/Archivos_csv/DatosModelo/DatosConLimpieza/validation_data_normalizada_sindup.csv')
#Dataser Reducido
#train = pd.read_csv('/home/gico/Neumonia/Archivos_csv/DatosModelo/DatosConLimpieza/train_datos_sololimpieza.csv')
#validation = pd.read_csv('/home/gico/Neumonia/Archivos_csv/DatosModelo/DatosConLimpieza/validationdatos_sololimpieza.csv')

# Data Normalizado solo sin hallazgos
#train = pd.read_csv('/home/gico/Neumonia/Archivos_csv/DatosModelo/DatosNormalizadoSóloSinHallazgos/train_data_normalizada_sindup.csv')
#validation = pd.read_csv('/home/gico/Neumonia/Archivos_csv/DatosModelo/DatosNormalizadoSóloSinHallazgos/validation_data_normalizada_sindup.csv')
#Dataser Reducido
#train = pd.read_csv('/home/gico/Neumonia/Archivos_csv/DatosModelo/DatosNormalizadoSóloSinHallazgos/train_datos_solo_sinhallazgo.csv')
#validation = pd.read_csv('/home/gico/Neumonia/Archivos_csv/DatosModelo/DatosNormalizadoSóloSinHallazgos/validationdatos_solo_sinhallazgo.csv')

# Data Normalizado reportes separados
train = pd.read_csv('/home/gico/Neumonia/Archivos_csv/DatosModelo/DatosConClusterNormalizadosReportesPorSeparado/train_data_normalizada_sindup.csv')
validation = pd.read_csv('/home/gico/Neumonia/Archivos_csv/DatosModelo/DatosConClusterNormalizadosReportesPorSeparado/validation_data_normalizada_sindup.csv')

# Data Normalizado reportes compuestos
#train = pd.read_csv('/home/gico/Neumonia/Archivos_csv/DatosModelo/DatosConClusterNormalizadosReportesCompuestos/train_data_normalizada_sindup.csv')
#validation = pd.read_csv('/home/gico/Neumonia/Archivos_csv/DatosModelo/DatosConClusterNormalizadosReportesCompuestos/validation_data_normalizada_sindup.csv')

diccionario = pd.read_csv('/home/gico/Neumonia/Archivos_csv/filtered_terms_dictionary.csv')

NameError: name 'pd' is not defined

In [2]:
# Preprocesar Datos
train = train.iloc[:, 0:]
validation = validation.iloc[:, 0:]
diccionario = diccionario.iloc[:, 0:]


# Convertir los reportes a minúsculas
train['Report'] = train['Report'].str.lower()
validation['Report'] = validation['Report'].str.lower()
diccionario['term'] = diccionario['term'].str.lower()

# añadir los tokenizer <start> y <end> a los dataset

train.Report      = '<start> ' + train.Report + ' <end>'
validation.Report = '<start> ' + validation.Report + ' <end>'
diccionario = '<start> ' + diccionario + ' <end>'

# Datos de train
image_train = [ ]
for col in tqdm(train.values):
    tensor = preprocess(col[1], path)
    image_train.append(tensor)

# Datos de validation
image_validation = [ ]
for col in tqdm(validation.values):
    tensor = preprocess(col[1], path)
    image_validation.append(tensor)

# Guardar los tensores en formato numpy
np.save("/home/gico/Neumonia/Tensores/train_image_features_3", image_train)
np.save("/home/gico/Neumonia/Tensores/validation_image_features_3",  image_validation)

image_train = np.load('/home/gico/Neumonia/Tensores/train_image_features_3.npy')
image_validation = np.load('/home/gico/Neumonia/Tensores/validation_image_features_3.npy')

PROCESAMIENTO DE TEXTO

FUNCIONES PARA PROCESAMIENTO DE TEXTO

In [ ]:
def prepare_text_data(df, text_column):
    # Asegurar que las columnas de texto son de tipo string
    df[text_column] = df[text_column].astype(str)
    return df

def create_tokenizer(train_texts):
    # Definir el tokenizador con los filtros adecuados, excluyendo apostrofes para capturar contracciones como "don't"
    tokenizer = Tokenizer(filters='!"#$%&()*+,-./:;=?@[\\]^`{|}~\t\n')
    tokenizer.fit_on_texts(train_texts)
    return tokenizer

def tokenize_and_pad(texts, tokenizer, max_len):
    # Convertir los textos a secuencias y rellenar
    sequences = tokenizer.texts_to_sequences(texts)
    padded = pad_sequences(sequences, maxlen=int(max_len), padding='post', truncating='post')
    return padded

def get_max_length(train_sequences, percentile=95):
    # Ajuste dinámico del tamaño de secuencia basado en un percentil
    max = np.percentile([len(x) for x in train_sequences], percentile)
    return int(np.ceil(max)) + 1

def train_word2vec(texts, vector_size=300, window=5, min_count=1, workers=4):
    # Preparar el corpus de texto y entrenar el modelo Word2Vec
    list_sents = [nltk.word_tokenize(sent) for text in texts for sent in nltk.sent_tokenize(text)]
    w2v_model = Word2Vec(sentences=list_sents, vector_size=vector_size, window=window, min_count=min_count, workers=workers)
    return w2v_model

def create_embedding_matrix(word_index, w2v_model, vector_size=300):
    # Crear una matriz de pesos para las palabras en documentos de entrenamiento
    embedding_matrix = np.zeros((len(word_index) + 1, vector_size))
    for word, i in tqdm(word_index.items()):
        if word in w2v_model.wv:
            embedding_vector = w2v_model.wv[word]
            if embedding_vector is not None:
                embedding_matrix[i] = embedding_vector
    return embedding_matrix

In [ ]:
# Preparar los datos de texto
train = prepare_text_data(train, 'Report')
validation = prepare_text_data(validation, 'Report')
diccionario = prepare_text_data(diccionario, 'term')

# Tokenización y padding
#combined_texts = pd.concat([train['Report'], diccionario['term']])
tokenizer = create_tokenizer(train['Report'])
max_len = get_max_length(tokenizer.texts_to_sequences(train['Report']))
pad_train = tokenize_and_pad(train['Report'], tokenizer, max_len)
pad_val = tokenize_and_pad(validation['Report'], tokenizer, max_len)

# Crear diccionarios para mapear índices a palabras y viceversa
word_idx = tokenizer.word_index
idx_word = {v: k for k, v in word_idx.items()}

# Calcular el tamaño del vocabulario (agregando uno por el token de padding 0)
vocab_size = len(word_idx) + 1

# Entrenamiento del modelo FastText
#texts = combined_texts.tolist()
w2v_model = train_word2vec(train['Report'])

# Creación de la matriz de embedding
embedding_matrix = create_embedding_matrix(tokenizer.word_index, w2v_model)

100%|██████████| 121/121 [00:00<00:00, 59129.77it/s]


CREACION DE DATASET NECESARIOS

In [ ]:
# Definición de hiperparámetros y constantes
BATCH_SIZE = 32
SHUFFLE_BUFFER_SIZE = 1000
PREFETCH_BUFFER_SIZE = tf.data.experimental.AUTOTUNE

def prepare_datasets(image_train, pad_train, image_validation, pad_va):
    """
    Prepara datasets de entrenamiento y validación con imágenes, texto preprocesado y localizaciones.

    Args:
        image_train (np.array): Imágenes de entrenamiento.
        pad_train (np.array): Texto de entrenamiento preprocesado y tokenizado.
        loc_train (np.array): Localizaciones de entrenamiento codificadas.
        image_validation (np.array): Imágenes de validación.
        pad_val (np.array): Texto de validación preprocesado y tokenizado.
        loc_val (np.array): Localizaciones de validación codificadas.

    Returns:
        train_dataset (tf.data.Dataset): Dataset de entrenamiento.
        val_dataset (tf.data.Dataset): Dataset de validación.
    """
    # Dataset de entrenamiento
    train_dataset = tf.data.Dataset.from_tensor_slices((image_train, pad_train))
    train_dataset = train_dataset.shuffle(SHUFFLE_BUFFER_SIZE).batch(BATCH_SIZE).prefetch(PREFETCH_BUFFER_SIZE)

    # Dataset de validación
    val_dataset = tf.data.Dataset.from_tensor_slices((image_validation, pad_val))
    val_dataset = val_dataset.shuffle(SHUFFLE_BUFFER_SIZE).batch(BATCH_SIZE).prefetch(PREFETCH_BUFFER_SIZE)

    return train_dataset, val_dataset

# Ejemplo de uso de la función modificada
train_dataset, val_dataset = prepare_datasets(image_train, pad_train, image_validation, pad_val)


MODELOS ENCODER - DECODER

MODELO ENCODER Y FUNCION DE ATECION PARA ENCODER

In [ ]:
class SelfAttention(tf.keras.layers.Layer):
    def __init__(self, embedding_dim):
        super(SelfAttention, self).__init__()
        self.embedding_dim = embedding_dim
        self.query_dense = Dense(units=self.embedding_dim)  # Genera los queries
        self.key_dense = Dense(units=self.embedding_dim)    # Genera las keys
        self.value_dense = Dense(units=self.embedding_dim)  # Genera los values

    def call(self, x):
        query = self.query_dense(x)
        key = self.key_dense(x)
        value = self.value_dense(x)
        scores = tf.matmul(query, key, transpose_b=True) / tf.sqrt(tf.cast(self.embedding_dim, tf.float32))
        weights = tf.nn.softmax(scores, axis=-1)
        output = tf.matmul(weights, value)
        return output

class Encoder(tf.keras.Model):
    def __init__(self, embedding_dim, dropout_rate=0.3, seed=None, debug=False):
        super(Encoder, self).__init__()
        self.embedding_dim = embedding_dim
        self.debug = debug
        self.seed = seed if seed is not None else 40

        self.dense1 = Dense(units=self.embedding_dim, activation='relu',
                            kernel_initializer=GlorotUniform(seed=self.seed),
                            name='encoder_dense_layer_1')
        self.attention = SelfAttention(embedding_dim=self.embedding_dim)
        self.dropout1 = Dropout(dropout_rate, name='encoder_dropout_1')
        self.dense_final = Dense(units=self.embedding_dim, activation='relu',
                                 kernel_initializer=GlorotUniform(seed=self.seed),
                                 name='encoder_dense_layer_final')
        self.reshape = Reshape((81,2048))

    def call(self, inputs):
        print(inputs.shape)
        x = self.reshape(inputs)
        x = self.dense1(x)
        x = self.dropout1(x)
        x = self.attention(x)
        x = self.dense_final(x)
        x = tf.nn.relu(x)
        return x

MODELO DECODER Y FUNCION DE ATENCION PARA DECODER

In [ ]:
class BahdanauAttention(tf.keras.layers.Layer):
    def __init__(self, units):
        super(BahdanauAttention, self).__init__()
        self.W1 = tf.keras.layers.Dense(units)
        self.W2 = tf.keras.layers.Dense(units)
        self.V = tf.keras.layers.Dense(1)

    def call(self, features, hidden):

        hidden_with_time_axis = tf.expand_dims(hidden, 1)
        score = self.V(tf.nn.tanh(self.W1(features) + self.W2(hidden_with_time_axis)))

        attention_weights = tf.nn.softmax(score, axis=1)
        context_vector = attention_weights * features
        context_vector = tf.reduce_sum(context_vector, axis=1)

        return context_vector, attention_weights

In [ ]:
# Definicion clase Decoder

class Decoder(tf.keras.Model):
    def __init__(self, embedding_dim, lstm_units, vocab_size):
        super(Decoder, self).__init__()
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim, mask_zero=True,
                                                   weights=[embedding_matrix],
                                                   name='Decoder_embedding')
        self.bidir_lstm = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(
                                                            lstm_units,
                                                            return_sequences=True,
                                                            return_state=True,
                                                            dropout=0.3
                                                        )
                                                    )

        self.dense = tf.keras.layers.Dense(vocab_size,
                                           activation=None,
                                           kernel_initializer=tf.keras.initializers.glorot_uniform(seed=45),
                                           name='Decoder_dense'
                                        )
        self.attention = BahdanauAttention(lstm_units)
        self.concat = tf.keras.layers.Concatenate()
        self.batch_norm = tf.keras.layers.BatchNormalization()

    def call(self, x, features, forward_h, forward_c, backward_h, backward_c):
        x = self.embedding(x)
        state = self.concat([forward_h, backward_h])
        context_vector, attention_weights = self.attention(features, state)
        result = tf.concat([tf.expand_dims(context_vector, axis=1), x], axis=-1)

        # Note: initial_state for LSTM includes both hidden state and cell state
        lstm_out, forward_h, forward_c, backward_h, backward_c = self.bidir_lstm(result, initial_state=[forward_h, forward_c, backward_h, backward_c])
        lstm_out = self.batch_norm(lstm_out)
        outputs = tf.reshape(lstm_out, (-1, lstm_out.shape[-1]))
        dense_out = self.dense(outputs)
        return dense_out, forward_h, forward_c, backward_h, backward_c, attention_weights

ENTRENAMIENTO

FUNCIONES DE PERDIDA, EXACTITUD Y OPTIMIZADOR

In [ ]:
# Ajustar la tasa de aprendizaje inicial y el decaimiento
optimizer = tf.keras.optimizers.Adam(learning_rate= 0.0001)
loss_obj = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True, reduction='none')
acc_obj = tf.keras.metrics.SparseCategoricalAccuracy()

def loss_func(real, pred):
    """Loss calculation"""
    loss_f = loss_obj(real, pred)
    return tf.reduce_mean(loss_f)

def acc_func(real, pred):
    """Accuracy calculation"""
    acc_f = acc_obj(real, pred)
    return tf.reduce_mean(acc_f)

In [ ]:
embedding_dim = 300
encoder = Encoder(embedding_dim = embedding_dim)

lstm_units = 1000
decoder = Decoder(embedding_dim = embedding_dim, lstm_units = lstm_units , vocab_size = vocab_size)

FUNCIONES DE TRAIN Y VAL

In [ ]:
@tf.function(autograph=True)
def train_step(tensor, target):
    loss = 0
    train_acc = 0

    # Inicialización de los estados ocultos y de celda para las LSTM bidireccionales
    forward_h = tf.zeros((target.shape[0], lstm_units))
    forward_c = tf.zeros((target.shape[0], lstm_units))
    backward_h = tf.zeros((target.shape[0], lstm_units))
    backward_c = tf.zeros((target.shape[0], lstm_units))

    dec_input = tf.expand_dims([tokenizer.word_index['<start>']] * target.shape[0], 1)

    with tf.GradientTape() as tape:
        features = encoder(tensor)
        for i in range(1, target.shape[1]):
            # Llamada al decoder con los estados ocultos y de celda
            predictions, forward_h, forward_c, backward_h, backward_c, _ = decoder(dec_input, features, forward_h, forward_c, backward_h, backward_c)
            loss += loss_func(target[:, i], predictions)
            train_acc += acc_func(target[:, i], predictions)
            dec_input = tf.expand_dims(target[:, i], 1)

    total_loss = loss / int(target.shape[1])
    total_acc = train_acc / int(target.shape[1])
    trainable_variables = encoder.trainable_variables + decoder.trainable_variables
    gradients = tape.gradient(loss, trainable_variables)
    optimizer.apply_gradients(zip(gradients, trainable_variables))

    return loss, total_loss, total_acc


@tf.function(autograph=True)
def val_step(tensor, target):
    loss_val = 0
    test_acc = 0
    forward_h = tf.zeros((target.shape[0], lstm_units))
    forward_c = tf.zeros((target.shape[0], lstm_units))
    backward_h = tf.zeros((target.shape[0], lstm_units))
    backward_c = tf.zeros((target.shape[0], lstm_units))
    dec_input = tf.expand_dims([tokenizer.word_index['<start>']] * target.shape[0], 1)
    features = encoder(tensor)

    for i in range(1, target.shape[1]):
        predictions_val, forward_h, forward_c, backward_h, backward_c, _ = decoder(dec_input, features, forward_h, forward_c, backward_h, backward_c)
        loss_val += loss_func(target[:, i], predictions_val)
        test_acc += acc_func(target[:, i], predictions_val)
        dec_input = tf.expand_dims(target[:, i], 1)

    total_loss_val = loss_val / int(target.shape[1])
    return loss_val, total_loss_val, test_acc

ENTRENAMIENTO DEL MODELO

ENTRENAMIENTO DATASET ENTRENAMIENTO

In [ ]:
tf.keras.backend.clear_session()
EPOCHS = 30

loss_plot_train = []
loss_plot_val = []
acc_plot_train = []
acc_plot_val = []
num_steps = 64

# Entrenamiento del modelo
for epoch in range(EPOCHS):
    start = time.time()
    total_loss_train = 0
    total_loss_val = 0
    total_acc_train = 0
    total_acc_val = 0
    acc_obj.reset_states()

    # Iterando sobre el dataset de entrenamiento
    for (batch, (img_tensor, target)) in enumerate(train_dataset):
        batch_loss, t_loss, t_acc = train_step(img_tensor, target)
        total_loss_train += t_loss
        total_acc_train += t_acc

    # Promedio de la precisión de entrenamiento
    avg_acc_train = total_acc_train / num_steps
    acc_plot_train.append(avg_acc_train)
    loss_plot_train.append(total_loss_train / num_steps)


    print(f'Epoch {epoch + 1} Train Loss: {total_loss_train / num_steps:.6f} Train Accuracy: {avg_acc_train:.6f}')
    print(f'Time taken for epoch: {time.time() - start} sec\n')

Input shape: (16, 1, 2048)
After dense1 shape: (16, 1, 300)
After attention shape: (16, 1, 300)
After dropout shape: (16, 1, 300)
Final output shape: (16, 1, 300)
Input shape: (16, 1, 2048)
After dense1 shape: (16, 1, 300)
After attention shape: (16, 1, 300)
After dropout shape: (16, 1, 300)
Final output shape: (16, 1, 300)
Epoch 1 Train Loss: 0.072767 Train Accuracy: 0.004767
Time taken for epoch: 179.29351449012756 sec

Epoch 2 Train Loss: 0.076128 Train Accuracy: 0.005405
Time taken for epoch: 0.2369518280029297 sec

Epoch 3 Train Loss: 0.072641 Train Accuracy: 0.001089
Time taken for epoch: 0.21404600143432617 sec

Epoch 4 Train Loss: 0.070576 Train Accuracy: 0.005586
Time taken for epoch: 0.20418524742126465 sec

Epoch 5 Train Loss: 0.069112 Train Accuracy: 0.005520
Time taken for epoch: 0.2056901454925537 sec



ENTRENAMIENTO DATASET VALIDACION

In [ ]:
# Entrenamiento del modelo
for epoch in range(EPOCHS):
    start = time.time()
    total_loss_train = 0
    total_loss_val = 0
    total_acc_train = 0
    total_acc_val = 0
    acc_obj.reset_states()

    # Iterando sobre el dataset de validación
    for (batch, (img_tensor, target)) in enumerate(val_dataset):
        batch_loss, t_loss, t_acc = val_step(img_tensor, target)
        total_loss_val += t_loss
        total_acc_val += t_acc

    # Promedio de la precisión de validación
    avg_acc_val = total_acc_val / num_steps
    acc_plot_val.append(avg_acc_val)
    loss_plot_val.append(total_loss_val / num_steps)

    print(f'Epoch {epoch + 1} ')
    print(f'Validation Loss: {total_loss_val / num_steps:.6f} Validation Accuracy: {avg_acc_val:.6f}')
    print(f'Time taken for epoch: {time.time() - start} sec\n')

Input shape: (4, 1, 2048)
After dense1 shape: (4, 1, 300)
After attention shape: (4, 1, 300)
After dropout shape: (4, 1, 300)
Final output shape: (4, 1, 300)
Epoch 1 
Validation Loss: 0.071580 Validation Accuracy: 0.253004
Time taken for epoch: 58.5090389251709 sec

Epoch 2 
Validation Loss: 0.071580 Validation Accuracy: 0.253004
Time taken for epoch: 0.07205653190612793 sec

Epoch 3 
Validation Loss: 0.071580 Validation Accuracy: 0.253004
Time taken for epoch: 0.06828713417053223 sec

Epoch 4 
Validation Loss: 0.071580 Validation Accuracy: 0.253004
Time taken for epoch: 0.07213568687438965 sec

Epoch 5 
Validation Loss: 0.071580 Validation Accuracy: 0.253004
Time taken for epoch: 0.09007883071899414 sec



INFORMACION DE ENTRENAMIENTO

GRAFICAS DE PERDIDA Y EXACTITUD

In [ ]:
# Gráfica combinada de pérdida de entrenamiento y validación
plt.figure(figsize=(10, 5))
plt.plot(loss_plot_train, label='Training Loss')
plt.plot(acc_plot_train, label='Training Accuracy')
plt.title('Loss y Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)
plt.savefig('/home/gico/Neumonia/Datos/Datos_entrenamiento/combined.png')
plt.close()

# Gráfica combinada de precisión de entrenamiento y validación
plt.figure(figsize=(10, 5))
plt.plot(loss_plot_val, label='Validation Loss')
plt.plot(acc_plot_val, label='Validation Accuracy')
plt.title('Training & Validation Accuracy per Epoch')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True)
plt.savefig('/home/gico/Neumonia/Datos/Datos_entrenamiento/combined_val.png')
plt.close()

CREACION PDF DE LAS GRAFICAS

In [ ]:
# Clase PDF para incluir gráficos en el documento
class PDF(FPDF):
    def header(self):
        self.set_font('Arial', 'B', 12)
        self.cell(0, 10, 'Training Report', 0, 1, 'C')

    def chapter_title(self, title):
        self.set_font('Arial', 'B', 12)
        self.cell(0, 10, title, 0, 1, 'L')

    def add_plot(self, img_file):
        self.image(img_file, x=10, w=180)  # Ajusta según necesidad

# Creación del documento PDF
pdf = PDF()
pdf.add_page()
pdf.set_font('Arial', 'B', 16)
pdf.cell(0, 10, 'Training and Validation Curves', 0, 1, 'C')

# Agregar las gráficas de pérdida combinada
pdf.chapter_title('Training')
pdf.add_plot('/home/gico/Neumonia/Datos/Datos_entrenamiento/combined.png')

# Agregar las gráficas de precisión combinada
pdf.chapter_title(' Validation')
pdf.add_plot('/home/gico/Neumonia/Datos/Datos_entrenamiento/combined_val.png')

# Guardar el PDF
pdf.output('/home/gico/Neumonia/Datos/Datos_entrenamiento/Training_Validation_Report.pdf')

''

EVALUACION DEL MODELO

FUNCION DE GENERACION

In [ ]:
def generate(image_path, path, top_k=10):
    """Generates captions for an image using Top-K sampling."""
    attention_shape =81
    max_len_output = max_len

    img_tensor_val = preprocess(image_path, path)
    features = encoder(img_tensor_val)
    forward_h = tf.zeros((1, lstm_units))
    forward_c = tf.zeros((1, lstm_units))
    backward_h = tf.zeros((1, lstm_units))
    backward_c = tf.zeros((1, lstm_units))
    dec_input = tf.expand_dims([tokenizer.word_index['<start>']], 0)

    result = []
    text = ""
    attention_plot = np.zeros((max_len_output, attention_shape))

    for i in range(max_len_output):
        predictions, forward_h, forward_c, backward_h, backward_c, attention_weights = decoder(dec_input, features, forward_h, forward_c, backward_h, backward_c)
        attention_plot[i] = tf.reshape(attention_weights, (-1)).numpy()

        # Aplicar Top-K Sampling
        top_k_values, top_k_indices = tf.nn.top_k(predictions, k=top_k, sorted=True)
        top_k_probs = tf.nn.softmax(top_k_values).numpy().flatten()
        chosen_index = np.random.choice(top_k_indices.numpy().flatten(), p=top_k_probs)

        predicted_word = tokenizer.index_word.get(chosen_index, '<unk>')
        if predicted_word == '<end>':
            break
        result.append(predicted_word)
        text += ' ' + predicted_word
        dec_input = tf.expand_dims([chosen_index], 0)

    attention_plot = attention_plot[:len(result), :]

    if not text.strip():
        return [], "No se generó texto", attention_plot

    return result, text, attention_plot

FUNCION DE EVALUACION

In [ ]:
def test_img_cap(index, path):
    # Seleccionar un punto de datos de validación
    img_data = validation.iloc[index]
    img_path = img_data['ImageID']
    actual_report = img_data['Report']

    actual_report = actual_report.replace('<start>', '').replace('<end>', '').strip()

    # Concatenar la ruta completa del archivo
    full_path = os.path.join(path, img_path)
    # Generar el reporte médico predicho
    result, text,  attention = generate(img_path, path)

    text = text.replace('<start>', '').replace('<end>', '').strip()

    # Cargar la imagen
    img = Image.open(full_path)
    result = result[:-1]

    # Calcular BLEU y ROUGE scores
    smoothie = SmoothingFunction().method4
    bleu_scores = {
        'BLEU-1': sentence_bleu([actual_report], text, weights=(1, 0, 0, 0), smoothing_function=smoothie),
        'BLEU-2': sentence_bleu([actual_report], text, weights=(0.5, 0.5, 0, 0), smoothing_function=smoothie),
        'BLEU-3': sentence_bleu([actual_report], text, weights=(0.33, 0.33, 0.33, 0), smoothing_function=smoothie),
        'BLEU-4': sentence_bleu([actual_report], text, weights=(0.25, 0.25, 0.25, 0.25), smoothing_function=smoothie)
    }
    rouge = Rouge()
    rouge_scores = rouge.get_scores(text, actual_report)

    return full_path, attention, actual_report, text, result, bleu_scores, rouge_scores

EVALUCION DE IMAGENES AL AZAR

In [ ]:
random_indices = random.sample(range(len(validation)), 10)
results = [test_img_cap(index, path) for index in random_indices]

1/1 [==============================] - 0s 446ms/step
Input shape: (1, 2048)
After dense1 shape: (1, 300)
After attention shape: (1, 300)
After dropout shape: (1, 300)
Final output shape: (1, 300)
1/1 [==============================] - 0s 29ms/step
Input shape: (1, 2048)
After dense1 shape: (1, 300)
After attention shape: (1, 300)
After dropout shape: (1, 300)
Final output shape: (1, 300)
1/1 [==============================] - 0s 25ms/step
Input shape: (1, 2048)
After dense1 shape: (1, 300)
After attention shape: (1, 300)
After dropout shape: (1, 300)
Final output shape: (1, 300)


In [ ]:
def save_image_with_report(index, image_path, actual_report, text, predicted_report, bleu_scores, rouge_scores, output_path):
    # Procesar reportes para remover los tokens
    actual_report_clean = actual_report.replace('<start>', '').replace('<end>', '').strip()
    predicted_report_clean = text.replace('<start>', '').replace('<end>', '').strip()

    img = Image.open(image_path)  # Abrir la imagen directamente sin manipulación de color
    img_array = np.array(img)     # Convertir la imagen a un arreglo de numpy

    # Crea una figura y dos subplots
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 6))

    # Muestra la imagen en el primer subplot
    ax1.imshow(img_array, cmap = 'bone')
    ax1.axis('off')  # Desactivar ejes

    # Prepara el texto del reporte y métricas
    report_and_metrics_text = (
        f"Image Name: {image_path}\n\n"
        f"Actual report: {actual_report_clean}\n\n"
        f"Predicted report: {predicted_report_clean}\n\n"
        f"BLEU-1: {bleu_scores['BLEU-1']:.4f}\n"
        f"BLEU-2: {bleu_scores['BLEU-2']:.4f}\n"
        f"BLEU-3: {bleu_scores['BLEU-3']:.4f}\n"
        f"BLEU-4: {bleu_scores['BLEU-4']:.4f}\n\n"
        f"ROUGE-1: {rouge_scores[0]['rouge-1']['f']:.4f}\n"
    )

    # Configura el segundo subplot para el texto
    ax2.axis('off')
    ax2.text(0.01, 0.5, report_and_metrics_text, transform=ax2.transAxes, fontsize=12, verticalalignment='center')

    # Guardar la figura en un archivo PNG
    plt.savefig(f'{output_path}/image_with_report_{index}.png', bbox_inches='tight')

    # Cierra la figura para liberar memoria
    plt.close(fig)

# Suponiendo que tienes una variable 'results' que ya contiene todos los datos necesarios para generar los informes
output_path = "/home/gico/Neumonia/Datos/Reportes_generados"
for index, (image_path, attention, actual_report, text, predicted_report, bleu_scores, rouge_scores) in enumerate(results):
    save_image_with_report(index, image_path, actual_report, text, predicted_report, bleu_scores, rouge_scores, output_path)

EVALUACION DEL DATASET DE VALIDACION

In [ ]:
validation_size = len(validation)
results = [test_img_cap(index, path) for index in range(validation_size)]

# Calcula las métricas medias
total_bleu1, total_bleu2, total_bleu3, total_bleu4, total_rouge_l = 0, 0, 0, 0, 0
for _, _, _, _, _, bleu_scores, rouge_scores in results:
    total_bleu1 += bleu_scores['BLEU-1']
    total_bleu2 += bleu_scores['BLEU-2']
    total_bleu3 += bleu_scores['BLEU-3']
    total_bleu4 += bleu_scores['BLEU-4']
    total_rouge_l += rouge_scores[0]['rouge-l']['f']

average_bleu1 = total_bleu1 / validation_size
average_bleu2 = total_bleu2 / validation_size
average_bleu3 = total_bleu3 / validation_size
average_bleu4 = total_bleu4 / validation_size
average_rouge_l = total_rouge_l / validation_size

print(f"Average BLEU-1: {average_bleu1:.4f}")
print(f"Average BLEU-2: {average_bleu2:.4f}")
print(f"Average BLEU-3: {average_bleu3:.4f}")
print(f"Average BLEU-4: {average_bleu4:.4f}")
print(f"Average ROUGE-L: {average_rouge_l:.4f}")

1/1 [==============================] - 0s 82ms/step
Input shape: (1, 2048)
After dense1 shape: (1, 300)
After attention shape: (1, 300)
After dropout shape: (1, 300)
Final output shape: (1, 300)
1/1 [==============================] - 0s 82ms/step
Input shape: (1, 2048)
After dense1 shape: (1, 300)
After attention shape: (1, 300)
After dropout shape: (1, 300)
Final output shape: (1, 300)
1/1 [==============================] - 0s 113ms/step
Input shape: (1, 2048)
After dense1 shape: (1, 300)
After attention shape: (1, 300)
After dropout shape: (1, 300)
Final output shape: (1, 300)
1/1 [==============================] - 0s 129ms/step
Input shape: (1, 2048)
After dense1 shape: (1, 300)
After attention shape: (1, 300)
After dropout shape: (1, 300)
Final output shape: (1, 300)
ROUGE scores structure: [{'rouge-1': {'r': 0.14285714285714285, 'p': 1.0, 'f': 0.24999999781250004}, 'rouge-2': {'r': 0.0, 'p': 0.0, 'f': 0.0}, 'rouge-l': {'r': 0.14285714285714285, 'p': 1.0, 'f': 0.24999999781250004}}